In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=4

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=4


In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
import pandas as pd
from ast import literal_eval

In [4]:
df = pd.read_csv('test_subset.csv')
df

,user,instance_id,text,sentiment,negative object_spans,negative spans,positive spans,positive object_spans
0,ekaterini@gmail.com,898,Nur Politiker wollen Krieg ! Was wenn keiner m...,negative,"['Politiker ', 'Krieg ']","['Was wenn keiner mehr Bock hat', 'zu sterben'...",...,...
1,ekaterini@gmail.com,833,Der Klimawandel ist ein völlig normaler Prozes...,positive,...,...,völlig normaler Prozess,Klimawandel
2,ekaterini@gmail.com,1428,Man will die Stadthalle für einen #Bürgerdialo...,negative,Demokraten,"['missliebige Partei ', 'Was denken die']",...,...
3,ekaterini@gmail.com,431,"Besonders beliebt ist auch ""Wir mussten währen...",negative,Corona,die während Corona auch gern auf die Maske ver...,...,...
4,ekaterini@gmail.com,1315,Die USA sollten sich mäßigen Israel zu ermahne...,positive,...,...,handelt moralisch richtig,"['Israel ', 'Israel ']"
...,...,...,...,...,...,...,...,...
295,ekaterini@gmail.com,1134,Auch Israel begeht Kriegsverbrechen an der Ziv...,negative,Israel,"['begeht Kriegsverbrechen', 'aber Kritik an Is...",...,...
296,ekaterini@gmail.com,127,Sogenannte Zivilisten? geht's noch?,negative,...,geht's noch?,...,...
297,ekaterini@gmail.com,366,"Ganz normal, dass wir in Zeiten der Klimakrise...",negative,Klimakrise,"['hypokapitalistischen', 'Nice one, Humanity',...",...,...
298,ekaterini@gmail.com,473,Kleine Nachhilfe: Ein paar hunderttausend Grün...,negative,Grüne,eine Klatsche bekommen habt,...,...


In [5]:
texts = list(df.text)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

#prompt = f"Classify the sentiment of each word in the texts into ONE of the three classes: neutral, negative or positive. Split the answer in two columns: Token and Label. Text: "
prompt = f"Classify the sentiment of each word in the texts into ONE of the three classes: neutral, negative or positive. Split the answer in a dictionary with two keys:Words and Labels. Text: "

answers = []
#text_list =[ "Ukraine deckt auf 😂😂😂😂😂\nDie mussten was sagen, nicht das man denkt der Staat wäre korrupt und es würde keinen Kampf dagegen geben ☝️\n\nKorrupt bleibt korrupt - auch die, da wird sich auch nichts änder 😆",'Nur Politiker wollen Krieg ! Was wenn keiner mehr Bock hat für diese 5 Politiker zu sterben warum drehen wir den Spieß nich einfach um ? 😕 frage für n Freund']
for text in texts:
    chat_template = [{"role" : "user", "content" : f"{prompt} {text}"}]

    chat_prompt = tokenizer.apply_chat_template(chat_template, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer.encode(chat_prompt, add_special_tokens=False, return_tensors="pt", max_length=512)
    outputs = model.generate(input_ids=inputs.to("cuda"), max_new_tokens = 512)
    answer = tokenizer.decode(outputs[0])
    answers.append(answer)
    #print(answer)
print(len(answers))

### The text generation here is not uniform before fine tuning like in gemma model, therefore quite hard to split each text and get the tokens and labels. But after fine-tuning, the results are formatted more uniformly based on the train data.

In [15]:
for answer in answers:
    print('labels',answer.split('{')[1].split('}')[0])

labels 

"Nur": "Neutral",
"Politiker": "Neutral",
"wollen": "Neutral",
"Krieg": "Negative",
"!": "Neutral",
"Was": "Neutral",
"wenn": "Neutral",
"keiner": "Neutral",
"mehr": "Neutral",
"Bock": "Neutral",
"diese": "Neutral",
"5": "Neutral",
"Politiker": "Neutral",
"zu": "Neutral",
"sterben": "Negative",
"warum": "Neutral",
"drehen": "Neutral",
"wir": "Neutral",
"den": "Neutral",
"Spieß": "Neutral",
"nich": "Neutral",
"einfach": "Neutral",
"um": "Neutral",
"😕": "Neutral",
"frage": "Neutral",
"für": "Neutral",
"n": "Neutral",
"Freund": "Neutral"


labels 
  "Words": [
    "Der",
    "Klimawandel",
    "ist",
    "ein",
    "völlig",
    "normaler",
    "Prozess",
    "den",
    "der",
    "Mensch",
    "nicht",
    "beeinflussen",
    "kann"
  ],
  "Labels": [
    "Neutral",
    "Neutral",
    "Neutral",
    "Neutral",
    "Neutral",
    "Neutral",
    "Neutral",
    "Neutral",
    "Neutral",
    "Neutral",
    "Neutral",
    "Neutral"
  ]

labels 
  "Words": 
labels 

"Besonders" : "neu